In [1]:
import requests
import os
import torch

In [2]:
import sys
sys.path.append('..')

In [45]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
from tqdm import tqdm
import importlib

In [4]:
import utils_clusters

In [5]:
import torch

In [ ]:
! wget -O dir.cla.scope.2.08-stable.txt https://zenodo.org/records/5829561/files/dir.cla.scope.2.08-stable.txt?download=1

In [6]:
df_scop = pd.read_csv("dir.cla.scope.2.08-stable.txt", sep='\t', skiprows=[0,1,2,3], header=None)

In [7]:
df_scop

,0,1,2,3,4,5
0,d1ux8a_,1ux8,A:,a.1.1.1,113449,"cl=46456,cf=46457,sf=46458,fa=46459,dm=46460,s..."
1,d1dlwa_,1dlw,A:,a.1.1.1,14982,"cl=46456,cf=46457,sf=46458,fa=46459,dm=46460,s..."
2,d1uvya_,1uvy,A:,a.1.1.1,100068,"cl=46456,cf=46457,sf=46458,fa=46459,dm=46460,s..."
3,d1dlya_,1dly,A:,a.1.1.1,14983,"cl=46456,cf=46457,sf=46458,fa=46459,dm=46460,s..."
4,d1uvxa_,1uvx,A:,a.1.1.1,100067,"cl=46456,cf=46457,sf=46458,fa=46459,dm=46460,s..."
...,...,...,...,...,...,...
344846,d2ehoc3,2eho,C:0-0,l.1.1.1,286555,"cl=310555,cf=310573,sf=310607,fa=310682,dm=310..."
344847,d2ehof3,2eho,F:0-0,l.1.1.1,286556,"cl=310555,cf=310573,sf=310607,fa=310682,dm=310..."
344848,d2ehog3,2eho,G:0-0,l.1.1.1,286557,"cl=310555,cf=310573,sf=310607,fa=310682,dm=310..."
344849,d2ehok3,2eho,K:0-0,l.1.1.1,286558,"cl=310555,cf=310573,sf=310607,fa=310682,dm=310..."


In [10]:
all_embedded_bert = os.listdir("/oak/stanford/groups/rbaltman/protbert_embeddings/post_pooling_seq_vectors/Pool_PaRTI/")
all_embedded_bert = [file[:-3] for file in all_embedded_bert]

In [12]:
all_embedded_esm = os.listdir("/oak/stanford/groups/rbaltman/esm_embeddings/esm2_t33_650M_uniprot/post_pooling_seq_vectors/Pool_PaRTI/")
all_embedded_esm = [file[:-3] for file in all_embedded_esm]

In [ ]:
df_acc_to_scop = {}
for acc in tqdm(all_embedded_bert):
    scop_labels = get_scop_labels_for_uniprot(acc)
    if scop_labels is None:
        continue
    df_acc_to_scop[acc] = scop_labels

In [14]:
import multiprocessing
from tqdm import tqdm

# Define the worker function
def process_acc(acc):
    try:
        # Check if the acc is already in the df_acc_to_scop dictionary
        if acc in df_acc_to_scop:
            return (acc, df_acc_to_scop[acc])

        # Fetch scop labels
        scop_labels = get_scop_labels_for_uniprot(acc)
        if scop_labels is None:
            return (acc, None)
        
        return (acc, scop_labels)

    except Exception as e:
        # Log the error or print it (optional)
        print(f"Error processing {acc}: {e}")
        # Return None for this acc to avoid crashing the entire process
        return (acc, None)

# Function to initialize the dictionary using multiprocessing
def create_dict_with_multiprocessing(all_embedded_esm):
    # Number of available CPUs
    num_cpus = multiprocessing.cpu_count()
    print(f"using {num_cpus} cpus")

    # Creating a pool of workers
    with multiprocessing.Pool(num_cpus) as pool:
        # Map the function over all elements and create an iterator to collect results
        results = pool.imap(process_acc, all_embedded_esm)
        
        # Initialize progress bar
        results_with_progress = tqdm(results, total=len(all_embedded_esm))
        
        # Collect results into a dictionary
        df_acc_to_scop_esm = {acc: scop_labels for acc, scop_labels in results_with_progress if scop_labels is not None}
        
    return df_acc_to_scop_esm

In [37]:
def filter_by_frequency(df, column_name, threshold):
    """
    Keeps rows in the DataFrame where the value in 'column_name' appears more than 'threshold' times.
    
    Parameters:
    - df (pd.DataFrame): The input DataFrame.
    - column_name (str): The column to check for frequency.
    - threshold (int): The minimum number of occurrences required to keep the row.

    Returns:
    - pd.DataFrame: Filtered DataFrame.
    """
    value_counts = df[column_name].value_counts()
    valid_values = value_counts[value_counts >= threshold].index
    filtered_df = df[df[column_name].isin(valid_values)].reset_index().drop(['index'], axis=1)
    return filtered_df

In [ ]:
df_acc_to_scop_esm = create_dict_with_multiprocessing(all_embedded_esm)

In [16]:
# Usage
df_acc_to_scop = torch.load("df_acc_to_scop.pt")
df_acc_to_scop_esm = torch.load("df_acc_to_scop_1to1_esm.pt")

In [16]:
torch.save(df_acc_to_scop, "df_acc_to_scop.pt")

In [18]:
len(df_acc_to_scop), len(df_acc_to_scop_esm)

(4725, 2963)

In [17]:
df_acc_to_scop_1to1 = {}
for acc in df_acc_to_scop:
    if len(df_acc_to_scop[acc]) == 1:
        df_acc_to_scop_1to1[acc] = df_acc_to_scop[acc][0]

In [17]:
df_acc_to_scop_1to1_esm = {}
for acc in df_acc_to_scop_esm:
    if len(df_acc_to_scop_esm[acc]) == 1:
        df_acc_to_scop_1to1_esm[acc] = df_acc_to_scop_esm[acc][0]

In [19]:
len(df_acc_to_scop_1to1)

1893

In [144]:
len(df_acc_to_scop_1to1_esm)

2963

In [20]:
torch.save(df_acc_to_scop_1to1, "df_acc_to_scop_1to1.pt")

In [18]:
torch.save(df_acc_to_scop_1to1_esm, "df_acc_to_scop_1to1_esm.pt")

In [19]:
importlib.reload(utils_clusters)

<module 'utils_clusters' from '/oak/stanford/groups/rbaltman/alptartici/pooling_work/pooling_performances/dimensionality_reduction_analysis/utils_clusters.py'>

In [68]:
df_acc_to_scop_1to1_protbert = torch.load("df_acc_to_scop_1to1.pt")

In [20]:
df_acc_to_scop_1to1_esm = torch.load("df_acc_to_scop_1to1_esm.pt")

In [69]:
# Convert the dictionary to a DataFrame
df_protbert = pd.DataFrame(list(df_acc_to_scop_1to1.items()), columns=['uniprot_acc', 'level_4'])

# Split 'level_4' into 'level_1', 'level_2', 'level_3'
df_protbert['level_1'] = df_protbert['level_4'].apply(lambda x: x.split('.')[0])  # First part (e.g., 'c')
df_protbert['level_2'] = df_protbert['level_4'].apply(lambda x: '.'.join(x.split('.')[:2]))  # First two parts (e.g., 'c.55')
df_protbert['level_3'] = df_protbert['level_4'].apply(lambda x: '.'.join(x.split('.')[:3]))  # First three parts (e.g., 'c.55.1')

In [22]:
# Convert the dictionary to a DataFrame
df_esm = pd.DataFrame(list(df_acc_to_scop_1to1_esm.items()), columns=['uniprot_acc', 'level_4'])

# Split 'level_4' into 'level_1', 'level_2', 'level_3'
df_esm['level_1'] = df_esm['level_4'].apply(lambda x: x.split('.')[0])  # First part (e.g., 'c')
df_esm['level_2'] = df_esm['level_4'].apply(lambda x: '.'.join(x.split('.')[:2]))  # First two parts (e.g., 'c.55')
df_esm['level_3'] = df_esm['level_4'].apply(lambda x: '.'.join(x.split('.')[:3]))  # First three parts (e.g., 'c.55.1')

In [54]:
df_esm = df_esm[df_esm['uniprot_acc'].isin(accs_PP_esm_cosine)]

In [55]:
df_esm_30Plus = filter_by_frequency(df_esm, 'level_2', 30)

In [56]:
df_esm_30Plus

,uniprot_acc,level_4,level_1,level_2,level_3
0,Q8IVW4,d.144.1.0,d,d.144,d.144.1
1,Q9Y616,d.144.1.0,d,d.144,d.144.1
2,P15531,d.58.6.1,d,d.58,d.58.6
3,O60684,a.118.1.0,a,a.118,a.118.1
4,Q6P1L8,a.28.1.0,a,a.28,a.28.1
...,...,...,...,...,...
1237,Q924K8,a.4.1.3,a,a.4,a.4.1
1238,Q8TE67,b.34.2.0,b,b.34,b.34.2
1239,Q9NVR0,a.118.17.0,a,a.118,a.118.17
1240,Q92541,b.34.21.1,b,b.34,b.34.21


# compute MRR and Precision@10

## ESM

In [58]:
dists_meanpooled_esm_cosine = np.load('pairwise_distances/mean_pooled_cosine_distances_esm.npy')
accs_meanpooled_esm_cosine = np.load('pairwise_distances/mean_pooled_cosine_accessions_esm.npy')
dists_maxpooled_esm_cosine = np.load('pairwise_distances/max_pooled_cosine_distances_esm.npy')
accs_maxpooled_esm_cosine = np.load('pairwise_distances/max_pooled_cosine_accessions_esm.npy')
dists_clspooled_esm_cosine = np.load('pairwise_distances/cls_pooled_cosine_distances_esm.npy')
accs_clspooled_esm_cosine = np.load('pairwise_distances/cls_pooled_cosine_accessions_esm.npy')
dists_sumpooled_esm_cosine = np.load('pairwise_distances/sum_pooled_cosine_distances_esm.npy')
accs_sumpooled_esm_cosine = np.load('pairwise_distances/sum_pooled_cosine_accessions_esm.npy')

In [46]:
dists_PP_esm_cosine = np.load('pairwise_distances/Pool_PaRTI_cosine_distances_esm.npy')
accs_PP_esm_cosine = np.load('pairwise_distances/Pool_PaRTI_cosine_accessions_esm.npy')

In [57]:
utils_clusters.compute_subsample_metrics(
    dist_matrix = dists_PP_esm_cosine,
    accessions = accs_PP_esm_cosine, 
    df = df_esm_30Plus, 
    accession_col = 'uniprot_acc', 
    label_col = 'level_2',
    subsample_size=30,
    subsample_time=50,
    K=5
)   

{'mean_reciprocal_rank': {'mean': 0.6776992116038655,
  'ste': 0.00026199545511663064},
 'precision_at_k': {'mean': 0.4504561403508772, 'ste': 0.00023131084959697383}}

In [59]:
utils_clusters.compute_subsample_metrics(
    dist_matrix = dists_meanpooled_esm_cosine,
    accessions = accs_meanpooled_esm_cosine, 
    df = df_esm_30Plus, 
    accession_col = 'uniprot_acc', 
    label_col = 'level_2',
    subsample_size=30,
    subsample_time=50,
    K=5
)   

{'mean_reciprocal_rank': {'mean': 0.6547400353198479,
  'ste': 0.00024277625937502097},
 'precision_at_k': {'mean': 0.42017543859649126,
  'ste': 0.00024322547857013165}}

In [60]:
utils_clusters.compute_subsample_metrics(
    dist_matrix = dists_maxpooled_esm_cosine,
    accessions = accs_maxpooled_esm_cosine, 
    df = df_esm_30Plus, 
    accession_col = 'uniprot_acc', 
    label_col = 'level_2',
    subsample_size=30,
    subsample_time=50,
    K=5
)   

{'mean_reciprocal_rank': {'mean': 0.6751575882168472,
  'ste': 0.0003270272936765675},
 'precision_at_k': {'mean': 0.44674385964912255,
  'ste': 0.00027428199223610033}}

In [61]:
utils_clusters.compute_subsample_metrics(
    dist_matrix = dists_clspooled_esm_cosine,
    accessions = accs_clspooled_esm_cosine, 
    df = df_esm_30Plus, 
    accession_col = 'uniprot_acc', 
    label_col = 'level_2',
    subsample_size=30,
    subsample_time=50,
    K=5
)   

{'mean_reciprocal_rank': {'mean': 0.6674797647309386,
  'ste': 0.0002641973553181234},
 'precision_at_k': {'mean': 0.44166315789473676, 'ste': 0.0002401387103002601}}

In [62]:
utils_clusters.compute_subsample_metrics(
    dist_matrix = dists_sumpooled_esm_cosine,
    accessions = accs_sumpooled_esm_cosine, 
    df = df_esm_30Plus, 
    accession_col = 'uniprot_acc', 
    label_col = 'level_2',
    subsample_size=30,
    subsample_time=50,
    K=5
)   

{'mean_reciprocal_rank': {'mean': 0.6591672931640615,
  'ste': 0.0003298917219252022},
 'precision_at_k': {'mean': 0.42361403508771933, 'ste': 0.0002701973105751499}}

#### k = 3

In [63]:
utils_clusters.compute_subsample_metrics(
    dist_matrix = dists_PP_esm_cosine,
    accessions = accs_PP_esm_cosine, 
    df = df_esm_30Plus, 
    accession_col = 'uniprot_acc', 
    label_col = 'level_2',
    subsample_size=30,
    subsample_time=50,
    K=3
)   

{'mean_reciprocal_rank': {'mean': 0.6815057566073522,
  'ste': 0.0002737488178113593},
 'precision_at_k': {'mean': 0.5076842105263156, 'ste': 0.0003080268344829769}}

In [64]:
utils_clusters.compute_subsample_metrics(
    dist_matrix = dists_meanpooled_esm_cosine,
    accessions = accs_meanpooled_esm_cosine, 
    df = df_esm_30Plus, 
    accession_col = 'uniprot_acc', 
    label_col = 'level_2',
    subsample_size=30,
    subsample_time=50,
    K=3
)   

{'mean_reciprocal_rank': {'mean': 0.6567428127204696,
  'ste': 0.00022599786376803694},
 'precision_at_k': {'mean': 0.4738128654970758, 'ste': 0.00022417474020938112}}

In [65]:
utils_clusters.compute_subsample_metrics(
    dist_matrix = dists_maxpooled_esm_cosine,
    accessions = accs_maxpooled_esm_cosine, 
    df = df_esm_30Plus, 
    accession_col = 'uniprot_acc', 
    label_col = 'level_2',
    subsample_size=30,
    subsample_time=50,
    K=3
)   

{'mean_reciprocal_rank': {'mean': 0.6744087671070734,
  'ste': 0.0002524576481630603},
 'precision_at_k': {'mean': 0.49989473684210495, 'ste': 0.0002485999510022396}}

In [66]:
utils_clusters.compute_subsample_metrics(
    dist_matrix = dists_clspooled_esm_cosine,
    accessions = accs_clspooled_esm_cosine, 
    df = df_esm_30Plus, 
    accession_col = 'uniprot_acc', 
    label_col = 'level_2',
    subsample_size=30,
    subsample_time=50,
    K=3
)   

{'mean_reciprocal_rank': {'mean': 0.6668544875529819,
  'ste': 0.0003223593892512703},
 'precision_at_k': {'mean': 0.495707602339181, 'ste': 0.0002898000168963085}}

In [67]:
utils_clusters.compute_subsample_metrics(
    dist_matrix = dists_sumpooled_esm_cosine,
    accessions = accs_sumpooled_esm_cosine, 
    df = df_esm_30Plus, 
    accession_col = 'uniprot_acc', 
    label_col = 'level_2',
    subsample_size=30,
    subsample_time=50,
    K=3
)   

{'mean_reciprocal_rank': {'mean': 0.6587637332222459,
  'ste': 0.00024859062523913014},
 'precision_at_k': {'mean': 0.4773216374269004, 'ste': 0.00027097464134082094}}

# protBERT

In [70]:
dists_PP_protbert_cosine = np.load('pairwise_distances/Pool_PaRTI_cosine_distances_protbert.npy')
accs_PP_protbert_cosine = np.load('pairwise_distances/Pool_PaRTI_cosine_accessions_protbert.npy')

In [71]:
dists_meanpooled_protbert_cosine = np.load('pairwise_distances/mean_pooled_cosine_distances_protbert.npy')
accs_meanpooled_protbert_cosine = np.load('pairwise_distances/mean_pooled_cosine_accessions_protbert.npy')
dists_maxpooled_protbert_cosine = np.load('pairwise_distances/max_pooled_cosine_distances_protbert.npy')
accs_maxpooled_protbert_cosine = np.load('pairwise_distances/max_pooled_cosine_accessions_protbert.npy')
dists_sumpooled_protbert_cosine = np.load('pairwise_distances/sum_pooled_cosine_distances_protbert.npy')
accs_sumpooled_protbert_cosine = np.load('pairwise_distances/sum_pooled_cosine_accessions_protbert.npy')
dists_clspooled_protbert_cosine = np.load('pairwise_distances/cls_pooled_cosine_distances_protbert.npy')
accs_clspooled_protbert_cosine = np.load('pairwise_distances/cls_pooled_cosine_accessions_protbert.npy')

In [72]:
df_protbert = df_protbert[df_protbert['uniprot_acc'].isin(accs_PP_protbert_cosine)]

In [74]:
df_protbert_30Plus = filter_by_frequency(df_protbert, 'level_2', 30)

In [77]:
utils_clusters.compute_subsample_metrics(
    dist_matrix = dists_PP_protbert_cosine,
    accessions = accs_PP_protbert_cosine, 
    df = df_protbert_30Plus, 
    accession_col = 'uniprot_acc', 
    label_col = 'level_2',
    subsample_size=30,
    subsample_time=50,
    K=3
)   

{'mean_reciprocal_rank': {'mean': 0.5487297805984456,
  'ste': 0.00030996908150868386},
 'precision_at_k': {'mean': 0.34211111111111114, 'ste': 0.000271894234258049}}

In [78]:
utils_clusters.compute_subsample_metrics(
    dist_matrix = dists_meanpooled_protbert_cosine,
    accessions = accs_meanpooled_protbert_cosine, 
    df = df_protbert_30Plus, 
    accession_col = 'uniprot_acc', 
    label_col = 'level_2',
    subsample_size=30,
    subsample_time=50,
    K=3
)   

{'mean_reciprocal_rank': {'mean': 0.5326954626609222,
  'ste': 0.00032950552325845784},
 'precision_at_k': {'mean': 0.3275370370370369, 'ste': 0.0002959444670407211}}

In [79]:
utils_clusters.compute_subsample_metrics(
    dist_matrix = dists_maxpooled_protbert_cosine,
    accessions = accs_maxpooled_protbert_cosine, 
    df = df_protbert_30Plus, 
    accession_col = 'uniprot_acc', 
    label_col = 'level_2',
    subsample_size=30,
    subsample_time=50,
    K=3
)   

{'mean_reciprocal_rank': {'mean': 0.5035400794164793,
  'ste': 0.0003268925396063341},
 'precision_at_k': {'mean': 0.2938518518518517, 'ste': 0.00025146375326155865}}

In [81]:
utils_clusters.compute_subsample_metrics(
    dist_matrix = dists_clspooled_protbert_cosine,
    accessions = accs_clspooled_protbert_cosine, 
    df = df_protbert_30Plus, 
    accession_col = 'uniprot_acc', 
    label_col = 'level_2',
    subsample_size=30,
    subsample_time=50,
    K=3
)   

{'mean_reciprocal_rank': {'mean': 0.461157953067486,
  'ste': 0.0003339001362025136},
 'precision_at_k': {'mean': 0.24755555555555545, 'ste': 0.0002826670937181336}}

In [80]:
utils_clusters.compute_subsample_metrics(
    dist_matrix = dists_sumpooled_protbert_cosine,
    accessions = accs_sumpooled_protbert_cosine, 
    df = df_protbert_30Plus, 
    accession_col = 'uniprot_acc', 
    label_col = 'level_2',
    subsample_size=30,
    subsample_time=50,
    K=3
)   

{'mean_reciprocal_rank': {'mean': 0.5307504103921161,
  'ste': 0.00037107027351998795},
 'precision_at_k': {'mean': 0.32416666666666655, 'ste': 0.0003035964128460748}}

In [61]:
pooling = 'PR_max_of_max_attn_no_prune'
PLM = 'protbert'
metric = "cosine"
level = 'level_1'
utils_clusters.evaluate_embeddings(df_filt_for_level1, 'uniprot_acc', level, PLM, pooling, distance_metric=metric)

Processing UniProt Accessions: 100%|██████████| 1108/1108 [00:01<00:00, 1010.84it/s]
/home/groups/rbaltman/tartici/miniconda3/envs/substrateSpec1/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


{'Silhouette Score': -0.038206060204040514,
 'Adjusted Rand Index': 0.025280583661280175,
 'Weighted Intra-class Mean Distance': 0.00040794204196289426,
 'Weighted Inter-class Mean Distance': 0.0010868320074448176,
 'ratio_of_inter_to_intra': 2.66418239761538}

In [60]:
pooling = 'mean_pooled'
PLM = 'protbert'
metric = "cosine"
level = 'level_1'
utils_clusters.evaluate_embeddings(df_filt_for_level1, 'uniprot_acc', level, PLM, pooling, distance_metric=metric)

Processing UniProt Accessions: 100%|██████████| 1108/1108 [00:01<00:00, 1047.10it/s]
/home/groups/rbaltman/tartici/miniconda3/envs/substrateSpec1/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


{'Silhouette Score': -0.03960645,
 'Adjusted Rand Index': 0.02433929253586319,
 'Weighted Intra-class Mean Distance': 0.0004131947938658522,
 'Weighted Inter-class Mean Distance': 0.001077105902027868,
 'ratio_of_inter_to_intra': 2.606775104667851}

In [62]:
pooling = 'max_pooled'
PLM = 'protbert'
metric = "cosine"
level = 'level_1'
utils_clusters.evaluate_embeddings(df_filt_for_level1, 'uniprot_acc', level, PLM, pooling, distance_metric=metric)

Processing UniProt Accessions: 100%|██████████| 1108/1108 [00:01<00:00, 1003.33it/s]
/home/groups/rbaltman/tartici/miniconda3/envs/substrateSpec1/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


{'Silhouette Score': -0.053922433,
 'Adjusted Rand Index': 0.032187936427173734,
 'Weighted Intra-class Mean Distance': 0.00011347800155516987,
 'Weighted Inter-class Mean Distance': 0.0003231623649396561,
 'ratio_of_inter_to_intra': 2.847797463039949}

In [63]:
pooling = 'cls_pooled'
PLM = 'protbert'
metric = "cosine"
level = 'level_1'
utils_clusters.evaluate_embeddings(df_filt_for_level1, 'uniprot_acc', level, PLM, pooling, distance_metric=metric)

Processing UniProt Accessions: 100%|██████████| 1108/1108 [00:01<00:00, 1026.24it/s]
/home/groups/rbaltman/tartici/miniconda3/envs/substrateSpec1/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


{'Silhouette Score': -0.047714263,
 'Adjusted Rand Index': 0.007679865432259322,
 'Weighted Intra-class Mean Distance': 0.00016011706929883093,
 'Weighted Inter-class Mean Distance': 0.0003816472641804014,
 'ratio_of_inter_to_intra': 2.3835513968103084}

In [64]:
pooling = 'PR_max_of_max_attn_no_prune'
PLM = 'protbert'
metric = "cosine"
level = 'level_2'
utils_clusters.evaluate_embeddings(df_filt_for_level1, 'uniprot_acc', level, PLM, pooling, distance_metric=metric)

Processing UniProt Accessions: 100%|██████████| 1108/1108 [00:01<00:00, 1002.30it/s]
/home/groups/rbaltman/tartici/miniconda3/envs/substrateSpec1/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


{'Silhouette Score': -0.32251402266333906,
 'Adjusted Rand Index': 0.048874106600442536,
 'Weighted Intra-class Mean Distance': 0.0035366331747871396,
 'Weighted Inter-class Mean Distance': 0.0285657807498176,
 'ratio_of_inter_to_intra': 8.07711157421264}

In [65]:
pooling = 'mean_pooled'
PLM = 'protbert'
metric = "cosine"
level = 'level_2'
utils_clusters.evaluate_embeddings(df_filt_for_level1, 'uniprot_acc', level, PLM, pooling, distance_metric=metric)

Processing UniProt Accessions: 100%|██████████| 1108/1108 [00:01<00:00, 1065.22it/s]
/home/groups/rbaltman/tartici/miniconda3/envs/substrateSpec1/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


{'Silhouette Score': -0.32520375,
 'Adjusted Rand Index': 0.05599724402734166,
 'Weighted Intra-class Mean Distance': 0.0036248677570447775,
 'Weighted Inter-class Mean Distance': 0.028718247005947852,
 'ratio_of_inter_to_intra': 7.9225640577191125}

In [66]:
pooling = 'max_pooled'
PLM = 'protbert'
metric = "cosine"
level = 'level_2'
utils_clusters.evaluate_embeddings(df_filt_for_level1, 'uniprot_acc', level, PLM, pooling, distance_metric=metric)

Processing UniProt Accessions: 100%|██████████| 1108/1108 [00:01<00:00, 1001.35it/s]
/home/groups/rbaltman/tartici/miniconda3/envs/substrateSpec1/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


{'Silhouette Score': -0.27340266,
 'Adjusted Rand Index': 0.048397537816689186,
 'Weighted Intra-class Mean Distance': 0.0009977530770223576,
 'Weighted Inter-class Mean Distance': 0.008804873041287084,
 'ratio_of_inter_to_intra': 8.824701465781384}

In [67]:
pooling = 'cls_pooled'
PLM = 'protbert'
metric = "cosine"
level = 'level_2'
utils_clusters.evaluate_embeddings(df_filt_for_level1, 'uniprot_acc', level, PLM, pooling, distance_metric=metric)

Processing UniProt Accessions: 100%|██████████| 1108/1108 [00:01<00:00, 1093.73it/s]
/home/groups/rbaltman/tartici/miniconda3/envs/substrateSpec1/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


{'Silhouette Score': -0.49648777,
 'Adjusted Rand Index': 0.02625884527312324,
 'Weighted Intra-class Mean Distance': 0.0015411555291329302,
 'Weighted Inter-class Mean Distance': 0.010500662940305886,
 'ratio_of_inter_to_intra': 6.813499832955643}

In [71]:
pooling = 'PR_max_of_max_attn_no_prune'
PLM = 'esm'
metric = "euclidean"
level = 'level_1'
utils_clusters.evaluate_embeddings(df_filt_for_level1, 'uniprot_acc', level, PLM, pooling, distance_metric=metric)

Processing UniProt Accessions: 100%|██████████| 1108/1108 [00:01<00:00, 1011.89it/s]
/home/groups/rbaltman/tartici/miniconda3/envs/substrateSpec1/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


{'Silhouette Score': -0.027976367508854034,
 'Adjusted Rand Index': 0.015110597498711183,
 'Weighted Intra-class Mean Distance': 0.010528567383684786,
 'Weighted Inter-class Mean Distance': 0.026356535869928466,
 'ratio_of_inter_to_intra': 2.503335440562495}

In [73]:
pooling = 'mean_pooled'
PLM = 'esm'
metric = "euclidean"
level = 'level_1'
utils_clusters.evaluate_embeddings(df_filt_for_level1, 'uniprot_acc', level, PLM, pooling, distance_metric=metric)

Processing UniProt Accessions: 100%|██████████| 1108/1108 [00:41<00:00, 26.79it/s]
/home/groups/rbaltman/tartici/miniconda3/envs/substrateSpec1/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


{'Silhouette Score': -0.036722366,
 'Adjusted Rand Index': 0.010321891501320875,
 'Weighted Intra-class Mean Distance': 0.009942976998543148,
 'Weighted Inter-class Mean Distance': 0.024698006146443817,
 'ratio_of_inter_to_intra': 2.483964928216427}

In [74]:
pooling = 'max_pooled'
PLM = 'esm'
metric = "euclidean"
level = 'level_1'
utils_clusters.evaluate_embeddings(df_filt_for_level1, 'uniprot_acc', level, PLM, pooling, distance_metric=metric)

Processing UniProt Accessions: 100%|██████████| 1108/1108 [00:40<00:00, 27.48it/s]
/home/groups/rbaltman/tartici/miniconda3/envs/substrateSpec1/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


{'Silhouette Score': -0.032044508,
 'Adjusted Rand Index': 0.040333023375864294,
 'Weighted Intra-class Mean Distance': 0.019658892555014285,
 'Weighted Inter-class Mean Distance': 0.05193480433670197,
 'ratio_of_inter_to_intra': 2.641797048911346}

In [75]:
pooling = 'cls_pooled'
PLM = 'esm'
metric = "euclidean"
level = 'level_1'
utils_clusters.evaluate_embeddings(df_filt_for_level1, 'uniprot_acc', level, PLM, pooling, distance_metric=metric)

Processing UniProt Accessions: 100%|██████████| 1108/1108 [00:37<00:00, 29.90it/s]
/home/groups/rbaltman/tartici/miniconda3/envs/substrateSpec1/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


{'Silhouette Score': -0.046801735,
 'Adjusted Rand Index': 0.004170928606612213,
 'Weighted Intra-class Mean Distance': 0.0048399784128565024,
 'Weighted Inter-class Mean Distance': 0.011301410309143004,
 'ratio_of_inter_to_intra': 2.3350125445028658}

In [76]:
pooling = 'PR_max_of_max_attn_no_prune'
PLM = 'esm'
metric = "cosine"
level = 'level_1'
utils_clusters.evaluate_embeddings(df_filt_for_level1, 'uniprot_acc', level, PLM, pooling, distance_metric=metric)

Processing UniProt Accessions: 100%|██████████| 1108/1108 [00:01<00:00, 955.27it/s] 
/home/groups/rbaltman/tartici/miniconda3/envs/substrateSpec1/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


{'Silhouette Score': -0.027976367508854034,
 'Adjusted Rand Index': 0.015110597498711183,
 'Weighted Intra-class Mean Distance': 0.00046114489185898995,
 'Weighted Inter-class Mean Distance': 0.0011423713614350849,
 'ratio_of_inter_to_intra': 2.477250386163666}

In [77]:
pooling = 'mean_pooled'
PLM = 'esm'
metric = "cosine"
level = 'level_1'
utils_clusters.evaluate_embeddings(df_filt_for_level1, 'uniprot_acc', level, PLM, pooling, distance_metric=metric)

Processing UniProt Accessions: 100%|██████████| 1108/1108 [00:01<00:00, 1004.41it/s]
/home/groups/rbaltman/tartici/miniconda3/envs/substrateSpec1/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


{'Silhouette Score': -0.036722366,
 'Adjusted Rand Index': 0.010321891501320875,
 'Weighted Intra-class Mean Distance': 0.00036527370194016365,
 'Weighted Inter-class Mean Distance': 0.0008849331405558774,
 'ratio_of_inter_to_intra': 2.42265768341801}

In [78]:
pooling = 'max_pooled'
PLM = 'esm'
metric = "cosine"
level = 'level_1'
utils_clusters.evaluate_embeddings(df_filt_for_level1, 'uniprot_acc', level, PLM, pooling, distance_metric=metric)

Processing UniProt Accessions: 100%|██████████| 1108/1108 [00:01<00:00, 1014.59it/s]
/home/groups/rbaltman/tartici/miniconda3/envs/substrateSpec1/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


{'Silhouette Score': -0.032044508,
 'Adjusted Rand Index': 0.040333023375864294,
 'Weighted Intra-class Mean Distance': 0.00018895097353804403,
 'Weighted Inter-class Mean Distance': 0.0005801154286577169,
 'ratio_of_inter_to_intra': 3.070190207519172}

In [80]:
pooling = 'cls_pooled'
PLM = 'esm'
metric = "cosine"
level = 'level_1'
utils_clusters.evaluate_embeddings(df_filt_for_level1, 'uniprot_acc', level, PLM, pooling, distance_metric=metric)

Processing UniProt Accessions: 100%|██████████| 1108/1108 [00:01<00:00, 1035.38it/s]
/home/groups/rbaltman/tartici/miniconda3/envs/substrateSpec1/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


{'Silhouette Score': -0.046801735,
 'Adjusted Rand Index': 0.004170928606612213,
 'Weighted Intra-class Mean Distance': 4.647748740699951e-05,
 'Weighted Inter-class Mean Distance': 0.00010692474537867655,
 'ratio_of_inter_to_intra': 2.3005706922653846}

In [20]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
PLMs = ['protbert', 'esm']
levels = ['level_1', 'level_2']
metrics = ["euclidean", "cosine"]
pooling_methods = ["PR_max_of_max_attn_no_prune", "mean_pooled", "max_pooled", "cls_pooled"]

In [100]:
filename = "scop_distance_ratios_with_abfgj_in_level1.txt"
with open(f"output/{filename}", "w") as f:
    for PLM in PLMs:
        for level in levels:
            for metric in metrics:
                for pooling in pooling_methods:
                    ratio = utils_clusters.evaluate_embeddings(df_filt_for_level1, 
                                                               'uniprot_acc', 
                                                               level, PLM, 
                                                               pooling, 
                                                               distance_metric=metric)['ratio_of_inter_to_intra']
                    f.write(f"{PLM}\t{level}\t{metric}\t{pooling}\t{ratio}\n")

In [101]:
level_1_to_keep_abcdfgj = ['a', 'b', 'c','d','f', 'g', 'j']
df_filt_for_level1_abcdfgj = df[df['level_1'].isin(level_1_to_keep_abcdfgj)]
df_filt_for_level1_abcdfgj = df_filt_for_level1_abcdfgj.reset_index()

In [102]:
filename = "scop_distance_ratios_with_abcdfgj_in_level1.txt"
with open(f"output/{filename}", "w") as f:
    for PLM in PLMs:
        for level in levels:
            for metric in metrics:
                for pooling in pooling_methods:
                    ratio = utils_clusters.evaluate_embeddings(df_filt_for_level1_abcdfgj, 
                                                               'uniprot_acc', 
                                                               level, PLM, 
                                                               pooling, 
                                                               distance_metric=metric)['ratio_of_inter_to_intra']
                    f.write(f"{PLM}\t{level}\t{metric}\t{pooling}\t{ratio}\n")

In [103]:
level_1_to_keep_abcdefgj = ['a', 'b', 'c','d', "e", 'f', 'g', 'j']
df_filt_for_level1_abcdefgj = df[df['level_1'].isin(level_1_to_keep_abcdefgj)]
df_filt_for_level1_abcdefgj = df_filt_for_level1_abcdefgj.reset_index()

In [105]:
filename = "scop_distance_ratios_with_abcdefgj_in_level1.txt"
with open(f"output/{filename}", "w") as f:
    for PLM in PLMs:
        for level in levels:
            for metric in metrics:
                for pooling in pooling_methods:
                    ratio = utils_clusters.evaluate_embeddings(df_filt_for_level1_abcdefgj, 
                                                               'uniprot_acc', 
                                                               level, PLM, 
                                                               pooling, 
                                                               distance_metric=metric)['ratio_of_inter_to_intra']
                    f.write(f"{PLM}\t{level}\t{metric}\t{pooling}\t{ratio}\n")

In [107]:
df_filt_for_level1_abcdefgj['level_1'].value_counts()

level_1
b    468
a    466
d    399
c    355
g    129
j     29
e     20
f     16
Name: count, dtype: int64

In [108]:
df_filt_for_level1_abcdefgj['level_2'].value_counts()

level_2
b.1      127
c.37      89
d.144     67
a.4       65
b.34      47
        ... 
d.209      1
a.194      1
b.95       1
c.110      1
c.84       1
Name: count, Length: 305, dtype: int64

In [110]:
len(df_filt_for_level1_abcdefgj)

1882

In [117]:
importlib.reload(utils_clusters)

<module 'utils_clusters' from '/oak/stanford/groups/rbaltman/alptartici/pooling_work/pooling_performances/dimensionality_reduction_analysis/utils_clusters.py'>

In [112]:
uniprot_col = 'uniprot_acc'
level = 'level_1'
PLM = 'protbert'
pooling = 'cls_pooled'
metric = "euclidean"

In [122]:
result = utils_clusters.evaluate_embeddings_for_dist_ratio(df_filt_for_level1_abcdefgj,
                                                  uniprot_col = uniprot_col,
                                                  label_col = level,
                                                  plm_type = PLM,
                                                  pooling_method = pooling,
                                                  distance_metric=metric,
                                                  verbose=True,
                                                  sampling_mode = True,
                                                  num_iterations = 2,
                                                  sample_size = 500,
                                                  min_samples_per_class=5
                                                 )

In [125]:
result["ratio_of_inter_to_intra_mean"]

1.8950032580632064

In [73]:
importlib.reload(utils_clusters)

<module 'utils_clusters' from '/oak/stanford/groups/rbaltman/alptartici/pooling_work/pooling_performances/dimensionality_reduction_analysis/utils_clusters.py'>

In [82]:
PLMs = ['protbert', 'esm']
#PLMs = ['esm']
levels = ['level_1', 'level_2']
metrics = ["euclidean", "cosine"]
pooling_methods = ["PR_max_of_max_attn_no_prune", "mean_pooled", "max_pooled", "cls_pooled"]
uniprot_col = 'uniprot_acc'

In [15]:
level_1_to_keep_abcdfgj = ['a', 'b', 'c','d','f', 'g', 'j']
df_filt_for_level1_abcdfgj = df[df['level_1'].isin(level_1_to_keep_abcdfgj)]
df_filt_for_level1_abcdfgj = df_filt_for_level1_abcdfgj.reset_index()
df_filt_for_level1_abcdfgj_esm = df_esm[df_esm['level_1'].isin(level_1_to_keep_abcdfgj)]
df_filt_for_level1_abcdfgj_esm = df_filt_for_level1_abcdfgj_esm.reset_index()


In [92]:
num_iter = 20
filename = f"scop_subsampled_distance_ratios_with_abfgj_in_level1_{num_iter}iterations.txt"
#filename = 'test_esm.txt'
with open(f"output/{filename}", "w") as f:
    for PLM in PLMs:
        if PLM == 'esm':
            df_to_use = df_filt_for_level1_abcdfgj_esm
        else:
            df_to_use = df_filt_for_level1_abcdfgj
        for level in levels:
            for metric in metrics:
                for pooling in pooling_methods:
                    result = utils_clusters.evaluate_embeddings_for_dist_ratio(df_to_use,
                                                  uniprot_col = uniprot_col,
                                                  label_col = level,
                                                  plm_type = PLM,
                                                  pooling_method = pooling,
                                                  distance_metric=metric,
                                                  verbose=False,
                                                  sampling_mode = True,
                                                  num_iterations = num_iter,
                                                  sample_size = int(len(df_to_use)//num_iter*2),
                                                  min_samples_per_class=5
                                                 )
                    mean = result["ratio_of_inter_to_intra_mean"]
                    std = result["ratio_of_inter_to_intra_std"]
                    ratios = result["ratios"]
                    f.write(f"{PLM}\t{level}\t{metric}\t{pooling}\t{mean} +/- {std} &{ratios}\n")

In [16]:
level_1_to_keep_abcdefgj = ['a', 'b', 'c','d', "e", 'f', 'g', 'j']
df_filt_for_level1_abcdefgj = df[df['level_1'].isin(level_1_to_keep_abcdefgj)]
df_filt_for_level1_abcdefgj = df_filt_for_level1_abcdefgj.reset_index()
df_filt_for_level1_abcdefgj_esm = df_esm[df_esm['level_1'].isin(level_1_to_keep_abcdefgj)]
df_filt_for_level1_abcdefgj_esm = df_filt_for_level1_abcdefgj_esm.reset_index()

In [90]:
num_iter = 20
filename = f"scop_subsampled_distance_ratios_with_abcdefgj_in_level1_{num_iter}iterations.txt"
with open(f"output/{filename}", "w") as f:
    for PLM in PLMs:
        if PLM == 'esm':
            df_to_use = df_filt_for_level1_abcdefgj_esm
        else:
            df_to_use = df_filt_for_level1_abcdefgj
        for level in levels:
            for metric in metrics:
                for pooling in pooling_methods:
                    result = utils_clusters.evaluate_embeddings_for_dist_ratio(df_to_use,
                                                  uniprot_col = uniprot_col,
                                                  label_col = level,
                                                  plm_type = PLM,
                                                  pooling_method = pooling,
                                                  distance_metric=metric,
                                                  verbose=True,
                                                  sampling_mode = True,
                                                  num_iterations = num_iter,
                                                  sample_size = int(len(df_to_use)//num_iter*2),
                                                  min_samples_per_class=5
                                                 )
                    mean = result["ratio_of_inter_to_intra_mean"]
                    std = result["ratio_of_inter_to_intra_std"]
                    ratios = result["ratios"]
                    f.write(f"{PLM}\t{level}\t{metric}\t{pooling}\t{mean} +/- {std} &{ratios}\n")

Embedding is None for Q92889
Embedding is None for P22415
Embedding is None for Q9BUI4
Embedding is None for Q9UKC9
Embedding is None for P68363
Embedding is None for Q9Y6Y1
Embedding is None for P13489
Embedding is None for P06132
Embedding is None for Q7L273
Embedding is None for P50440
Embedding is None for Q9HBH9
Embedding is None for P23193


Processing UniProt Accessions:   0%|          | 0/297 [00:00<?, ?it/s]

Embedding is None for Q9Y2J2
Embedding is None for P51531
Embedding is None for Q9NRD5


Embedding is None for Q96GD3
Embedding is None for P36941
Embedding is None for P62280
Embedding is None for P13489
Embedding is None for O15519
Embedding is None for P36639
Embedding is None for P52888
Embedding is None for Q9NWB1
Embedding is None for Q8N5K1


Processing UniProt Accessions:   0%|          | 0/297 [00:00<?, ?it/s]

...

Processing UniProt Accessions:  23%|██▎       | 255/1090 [00:00<00:00, 2548.99it/s]

Embedding is None for P02008
Embedding is None for Q9UKC9
Embedding is None for O75390
Embedding is None for Q9Y2J2
Embedding is None for P30566
Embedding is None for P82979
Embedding is None for Q9H074
Embedding is None for Q96EY1
Embedding is None for Q8IX01
Embedding is None for P18858
Embedding is None for P52657
Embedding is None for P61244
Embedding is None for P22415
Embedding is None for P55347
Embedding is None for P53814
Embedding is None for P43897
Embedding is None for P12429
Embedding is None for Q9UI12
Embedding is None for P51946
Embedding is None for P12277
Embedding is None for P68363
Embedding is None for P40394
Embedding is None for Q8TBB1
Embedding is None for Q9NSC5
Embedding is None for P13489
Embedding is None for O15519
Embedding is None for P55212


Processing UniProt Accessions:  93%|█████████▎| 1009/1090 [00:00<00:00, 2483.74it/s]

Embedding is None for Q71U36
Embedding is None for Q8WUI4
Embedding is None for O95671
Embedding is None for Q16831
Embedding is None for P50213
Embedding is None for O95630
Embedding is None for Q9Y6U3
Embedding is None for P50440
Embedding is None for P49773
Embedding is None for Q9Y237
Embedding is None for O43617
Embedding is None for O75569
Embedding is None for P18615
Embedding is None for P09543
Embedding is None for Q9Y2K5
Embedding is None for Q6Y7W6
Embedding is None for O75173
Embedding is None for P52888
Embedding is None for P16591
Embedding is None for Q92934
Embedding is None for Q07654
Embedding is None for O60711
Embedding is None for P23193
Embedding is None for Q9NXR8


Embedding is None for Q8N5K1


In [17]:
PLMs = ['protbert', 'esm']
#PLMs = ['esm']
levels = ['level_1', 'level_2']
metrics = ["euclidean", "cosine"]
pooling_methods = ["sum_pooled"]
uniprot_col = 'uniprot_acc'

In [18]:
num_iter = 20
filename = f"scop_subsampled_distance_ratios_with_abcdefgj_in_level1_{num_iter}iterations.txt"
with open(f"output/{filename}", "a") as f:
    f.write(f"\n\n\n")
    for PLM in PLMs:
        if PLM == 'esm':
            df_to_use = df_filt_for_level1_abcdefgj_esm
        else:
            df_to_use = df_filt_for_level1_abcdefgj
        for level in levels:
            for metric in metrics:
                for pooling in pooling_methods:
                    result = utils_clusters.evaluate_embeddings_for_dist_ratio(df_to_use,
                                                  uniprot_col = uniprot_col,
                                                  label_col = level,
                                                  plm_type = PLM,
                                                  pooling_method = pooling,
                                                  distance_metric=metric,
                                                  verbose=True,
                                                  sampling_mode = True,
                                                  num_iterations = num_iter,
                                                  sample_size = int(len(df_to_use)//num_iter*2),
                                                  min_samples_per_class=5
                                                 )
                    mean = result["ratio_of_inter_to_intra_mean"]
                    std = result["ratio_of_inter_to_intra_std"]
                    ratios = result["ratios"]
                    f.write(f"{PLM}\t{level}\t{metric}\t{pooling}\t{mean} +/- {std} &{ratios}\n")

Processing UniProt Accessions:  43%|████▎     | 127/297 [00:05<00:06, 27.29it/s]  

Skipping Q9Y3R0: Embedding file not found.


Processing UniProt Accessions:  90%|████████▉ | 266/297 [00:10<00:01, 26.47it/s]

Skipping Q9P2D1: Embedding file not found.


Processing UniProt Accessions:  99%|█████████▉| 294/297 [00:11<00:00, 30.51it/s]

...

Skipping Q8NHM5: Embedding file not found.
Skipping Q8IX01: Embedding file not found.
Skipping P42345: Embedding file not found.
Skipping P51531: Embedding file not found.
Skipping Q9HD67: Embedding file not found.
Skipping Q9C000: Embedding file not found.
Skipping Q8TD84: Embedding file not found.
Skipping O94966: Embedding file not found.


Skipping O95714: Embedding file not found.
Skipping Q5TCY1: Embedding file not found.
Skipping Q99683: Embedding file not found.
Skipping Q93009: Embedding file not found.
Skipping Q9HCK8: Embedding file not found.
Skipping Q9P2D1: Embedding file not found.
Skipping Q9Y6J0: Embedding file not found.
Skipping Q5VZK9: Embedding file not found.
Skipping O96028: Embedding file not found.
Skipping Q14839: Embedding file not found.


Processing UniProt Accessions:  43%|████▎     | 468/1090 [00:00<00:00, 2330.98it/s]

Skipping Q9Y2J2: Embedding file not found.
Skipping Q8NHM5: Embedding file not found.
Skipping Q8IX01: Embedding file not found.
Skipping Q96P70: Embedding file not found.
Skipping P42345: Embedding file not found.
Skipping O15164: Embedding file not found.
Skipping Q9C000: Embedding file not found.
Skipping O94966: Embedding file not found.
Skipping P30414: Embedding file not found.


Skipping O95714: Embedding file not found.
Skipping Q9P2D1: Embedding file not found.
Skipping Q9HCK8: Embedding file not found.
Skipping Q6Y7W6: Embedding file not found.
Skipping Q9Y6J0: Embedding file not found.
Skipping Q5VZK9: Embedding file not found.
Skipping O96028: Embedding file not found.
Skipping Q7Z3T8: Embedding file not found.
Skipping Q14839: Embedding file not found.


In [20]:
import re
from collections import defaultdict
import scipy.stats as stats

# Initialize a dictionary to hold the values for each PLM, level, metric, and pooling type
data_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(dict)))

# Function to process the file content
def process_file(file_name):
    with open(file_name, 'r') as f:
        for line in f:
            # Split the line based on the tab separator
            parts = line.strip().split('\t')
            if len(parts) == 5:
                plm, level, metric, pooling, mean_std_ratios = parts
                
                # Extract mean, std and ratios using regular expression
                match = re.match(r"(.+?) \+/- (.+?) &(.+)", mean_std_ratios)
                if match:
                    mean, std, ratios = match.groups()
                    
                    # Store the values in the dictionary
                    data_dict[plm][level][metric][pooling] = {
                        'mean': float(mean),
                        'std': float(std),
                        'ratios': ratios
                    }
    return data_dict

def run_significance_test(performance_data1, performance_data2, test_type='mannwhitney', alternative='greater'):

    performance_data1 = performance_data1[1:-1].split(', ')
    performance_data1 = [float(data) for data in performance_data1]

    performance_data2 = performance_data2[1:-1].split(', ')
    performance_data2 = [float(data) for data in performance_data2]
    
    if test_type == 'wilcoxon':
        # Perform Wilcoxon signed-rank test
        statistic, p_value = stats.wilcoxon(performance_data1, performance_data2)
    elif test_type == 'mannwhitney':
        # Perform Mann-Whitney U test
        statistic, p_value = stats.mannwhitneyu(performance_data1, performance_data2, alternative=alternative)
    else:
        raise ValueError("Invalid test_type. Choose either 'wilcoxon' or 'mannwhitney'.")
    
    return {
        'statistic': statistic,
        'p_value': p_value,
        'significant': p_value < 0.05,
        'test_type': test_type
    }

In [21]:
# Example file name provided by user
file_name = "scop_subsampled_distance_ratios_with_abcdefgj_in_level1_20iterations.txt"  # Replace with the actual file path
file_name = f"output/{file_name}"
# Process the file
data_abcdefgj = process_file(file_name)

### ProtBERT

#### level 1

In [105]:
data = data_abcdefgj
PLM = 'protbert'
level = 'level_1'
metric = 'euclidean'
method_1 = 'PR_max_of_max_attn_no_prune'

In [129]:
method_2 = 'mean_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 206.0,
 'p_value': 0.44086539586956214,
 'significant': False,
 'test_type': 'mannwhitney'}

In [130]:
method_2 = 'max_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 313.0,
 'p_value': 0.0011706363037467687,
 'significant': True,
 'test_type': 'mannwhitney'}

In [131]:
method_2 = 'cls_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 400.0,
 'p_value': 3.397807564086679e-08,
 'significant': True,
 'test_type': 'mannwhitney'}

#### level 2

In [24]:
data = data_abcdefgj
PLM = 'protbert'
level = 'level_2'
metric = 'euclidean'
method_1 = 'PR_max_of_max_attn_no_prune'

In [25]:
method_2 = 'mean_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 267.0,
 'p_value': 0.03602271523367195,
 'significant': True,
 'test_type': 'mannwhitney'}

In [26]:
method_2 = 'max_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 400.0,
 'p_value': 3.397807564086679e-08,
 'significant': True,
 'test_type': 'mannwhitney'}

In [27]:
method_2 = 'cls_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 400.0,
 'p_value': 3.397807564086679e-08,
 'significant': True,
 'test_type': 'mannwhitney'}

In [28]:
method_2 = 'sum_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 392.0,
 'p_value': 1.1088821650461217e-07,
 'significant': True,
 'test_type': 'mannwhitney'}

### ESM

#### level 1

In [141]:
data = data_abcdefgj
PLM = 'esm'
level = 'level_1'
metric = 'euclidean'
method_1 = 'PR_max_of_max_attn_no_prune'

In [142]:
method_2 = 'mean_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 283.0,
 'p_value': 0.01281963600440279,
 'significant': True,
 'test_type': 'mannwhitney'}

In [143]:
method_2 = 'max_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 158.0,
 'p_value': 0.8748516012482441,
 'significant': False,
 'test_type': 'mannwhitney'}

In [144]:
method_2 = 'cls_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 395.0,
 'p_value': 7.154265625721916e-08,
 'significant': True,
 'test_type': 'mannwhitney'}

#### level 2

In [22]:
data = data_abcdefgj
PLM = 'esm'
level = 'level_2'
metric = 'euclidean'
method_1 = 'PR_max_of_max_attn_no_prune'

In [146]:
method_2 = 'mean_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 394.0,
 'p_value': 8.285396466929393e-08,
 'significant': True,
 'test_type': 'mannwhitney'}

In [147]:
method_2 = 'max_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 398.0,
 'p_value': 4.586386355828241e-08,
 'significant': True,
 'test_type': 'mannwhitney'}

In [148]:
method_2 = 'cls_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 400.0,
 'p_value': 3.397807564086679e-08,
 'significant': True,
 'test_type': 'mannwhitney'}

In [23]:
method_2 = 'sum_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 369.0,
 'p_value': 2.5828894122485707e-06,
 'significant': True,
 'test_type': 'mannwhitney'}

In [151]:
data = data_abfgj
PLM = 'protbert'
level = 'level_1'
metric = 'euclidean'
method_1 = 'PR_max_of_max_attn_no_prune'

In [152]:
method_2 = 'mean_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 263.0,
 'p_value': 0.045453691625313236,
 'significant': True,
 'test_type': 'mannwhitney'}

In [153]:
method_2 = 'max_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 310.0,
 'p_value': 0.0015283146140873955,
 'significant': True,
 'test_type': 'mannwhitney'}

In [154]:
method_2 = 'cls_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 399.0,
 'p_value': 3.9490168333373314e-08,
 'significant': True,
 'test_type': 'mannwhitney'}

In [166]:
data = data_abfgj
PLM = 'esm'
level = 'level_1'
metric = 'euclidean'
method_1 = 'PR_max_of_max_attn_no_prune'

In [167]:
method_2 = 'mean_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 259.0,
 'p_value': 0.056775696491720214,
 'significant': False,
 'test_type': 'mannwhitney'}

In [168]:
method_2 = 'max_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 144.0,
 'p_value': 0.9367846913166159,
 'significant': False,
 'test_type': 'mannwhitney'}

In [169]:
method_2 = 'cls_pooled'
data_1 = data[PLM][level][metric][method_1]['ratios']
data_2 = data[PLM][level][metric][method_2]['ratios']
run_significance_test(data_1, data_2)

{'statistic': 397.0,
 'p_value': 5.322844918751244e-08,
 'significant': True,
 'test_type': 'mannwhitney'}